zero = r0
temp = r1
sp = r2
random = r16

In [9]:

function preprocess(source) {
	var lines = []
	var line =[]
	var temp=""
	var lock =false;
	for(var i=0;i<source.length;i++) {
		var c=source[i]
		if (!lock) 
		switch (c) {
			case ';':
				lock=true;
			case '\n':
				if (temp.length) line.push(temp)
				temp=""
				if (line.length)lines.push(line)
				line=[]
			break
			case ' ':
			case ',':
				if (temp.length) line.push(temp)
				temp=""
				break
			case '\t':
				break
			default:
				temp+=c;
			break
		}
		if (c=='\n') lock=false;
	}
	return lines;
}

In [12]:
preprocess(`
	mov     	         ax,bx;hi
	int 21h
`)

[ [ 'mov', 'ax', 'bx' ], [ 'int', '21h' ] ]


In [ ]:
function expand(line) { //do before push
	
}

In [ ]:
opcodes =  new Map();
opcodes["nop"]=0
opcodes["add"]=1
opcodes["sub"]=2
opcodes["mul"]=3
opcodes["mulh"]=4
opcodes["div"]=5
opcodes["rem"]=6
opcodes["not"]=7
opcodes["and"]=8
opcodes["or"]=9
opcodes["xor"]=10
opcodes["shl"]=
opcodes["shr"]=
opcodes["ror"]=
opcodes["rol"]=
opcodes["ashl"]=
opcodes[""]=
opcodes["add"]=
opcodes["sub"]=
opcodes["mul"]=
opcodes["mulh"]=
opcodes["div"]=
opcodes["rem"]=
opcodes["lui"]=
opcodes["and"]=
opcodes["or"]=
opcodes["xor"]=
opcodes["shl"]=
opcodes["shr"]=
opcodes["ror"]=
opcodes["rol"]=
opcodes["ashl"]=
opcodes["jr"]=
opcodes["bg"]=
opcodes["be"]=
opcodes["bge"]=
opcodes["bl"]=
opcodes["bne"]=
opcodes["ble"]=
opcodes["jal"]=
opcodes["load"]=
opcodes["loadi(adddr)"]=
opcodes["store"]=
opcodes["storei"]=
opcodes["In – reg"]=
opcodes["in"]=
opcodes["out- reg"]=
opcodes["out"]=
opcodes["."]=
opcodes[""]=
opcodes[""]=
opcodes[""]=
opcodes[""]=
opcodes[""]=
opcodes[""]=
opcodes[""]=
opcodes[""]=
opcodes[""]=
opcodes[""]=
opcodes[""]=
opcodes[""]=
opcodes[""]=
opcodes[""]=
opcodes["hlt"]=
